In [1]:
!pip install sentence-transformers

In [2]:
import torch
torch.cuda.is_available()

C:\Users\user\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
from datasets import load_dataset
import torch
from torch.utils.data import Dataset, DataLoader

dataset = load_dataset("tweet_eval", "sentiment")

Reusing dataset tweet_eval (C:\Users\user\.cache\huggingface\datasets\tweet_eval\sentiment\1.1.0\12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343)
100%|██████████| 3/3 [00:00<00:00, 752.43it/s]


In [4]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

# show data
def show_random_elements(dataset, num_examples=20):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [5]:
show_random_elements(dataset['train'])

,text,label
0,Dustin Johnson must have celebrated a bit too hard and too prematurely on Sat night. He finishes at -4 dropping 6 shots in the final 2 days.,neutral
1,@user Deathly Hallows Part II now become the 3rd High Grossing Movie in Worldwide All of time with $1.133M . Absolutely Amazing !,positive
2,Meanwhile David Price 2-0 with ERA under 1....,neutral
3,@user Ya our game in hand is v Bolton!!!!!. 6 pointer ha!..Playing Norwich last game of the season!.#hopeless,positive
4,@user David price done the same thing. And got his cunt kicked in a 2nd time,negative
5,Patriots fans\u002c Patriots Fourth And Two returns Friday at Noon EST. I hope you can take a listen.,neutral
6,@user On the holy occasion OF EID Here is wishing that May the blessing of Allah Light up your way And lead u to eternal,positive
7,"Okay, so anyone going to come see Jurassic World with me in IMAX?",positive
8,"The announcement ... was made on social media platform Google+ in April, with the news quickly spreading to other channels today.",neutral
9,"""We've seen all of the fights on recent Saturday nights...Rousey, Dana White, etc...but the real fight could be tonight in Vegas! #GOPDebate""",positive


In [6]:
dataset['train']
dataset['test']
dataset['validation']

Dataset({
    features: ['text', 'label'],
    num_rows: 2000
})

In [7]:
texts = []
labels = []

texts = dataset['train']['text'] + dataset['test']['text'] + dataset['validation']['text']
labels = dataset['train']['label'] + dataset['test']['label'] + dataset['validation']['label']

print(len(texts))
print(len(labels))

59899
59899


In [8]:
for index in range(len(texts)):
    texts[index] = texts[index].replace('…','...')
    texts[index] = texts[index].replace('\n', ' ')

In [9]:
temp = []

for text, label in zip(texts, labels):
    temp.append({'text' : text, 'label' : label})
print(len(temp))
print(temp[0]['text'])
print(temp[0]['label'])

59899
"QT @user In the original draft of the 7th book, Remus Lupin survived the Battle of Hogwarts. #HappyBirthdayRemusLupin"
2


In [10]:
sorted_dict = sorted(temp, key=(lambda x: x['text']))
print(sorted_dict[3]['text'])
print(sorted_dict[3]['label'])
sorted_positive = [x['text'] for x in sorted_dict if x['label'] == 2 and x['text'] <= 'z']
sorted_negative = [x['text'] for x in sorted_dict if x['label'] == 0 and x['text'] <= 'z']
random_negative = sorted_negative.copy()
random.shuffle(random_negative)
random.shuffle(sorted_positive) # 주석해제시 랜덤하게 pair
print(len(sorted_positive))

" #Refugees should just go home to #Syria " they said. #Aleppo
1
21011


In [11]:
final_dataset = []
index_2 = 0

for index in range(0, len(sorted_positive) - 1, 2):
    final_dataset.append({'sent0' : sorted_positive[index], 'sent1' : sorted_positive[index+1], 'hard_neg' : random_negative[index_2]})
    index_2 += 1
    
random.shuffle(final_dataset)
print(final_dataset[0])

{'sent0': 'Good morning neighbors... Tomorrow  is National Hot Dog Day.  Here are some fun facts about hot dogs.  It is an...', 'sent1': 'honestly i will die if Briana gives birth on February 1st that would be the funniest thing ever', 'hard_neg': '.@ChiOnwurah @user #DoNoEvil @user @user is this why #publiceducation is made 2 FAIL by ALEC+GOP?'}


In [12]:
print(final_dataset[7400]['sent0'])

"I love Israel and I will always support you, come what may.


In [13]:
import csv

# with open('tweeteval_sentiment.csv','w') as f:
#     w = csv.writer(f)
#     w.writerow(final_dataset[0].keys())
#     for i in range(len(finale_dataset)):
#         w.writerow(final_dataset[i].values())
        
labels = ['sent0','sent1','hard_neg']
try:
    with open('tweeteval_sentiment.csv', 'w',newline='',encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=labels)
        writer.writeheader()
        for elem in final_dataset:
            writer.writerow(elem)
except IOError:
    print("I/O error")